### Считываем данные и делаем препроцессинг, в результате которого получаем списки коннектомов для контрольной группы и группы пациентов ###

In [1]:
import numpy as np
import pandas as pd
import pathlib as plib

In [2]:
data_path = plib.Path("./data/OASIS3/ts_extracted")

In [3]:
controls_path = data_path / "controls/AAL"
patients_path = data_path / "patients/AAL"

In [4]:
alzheimer_features = [
    "Hippocampus_L",
    "Hippocampus_R",
    "ParaHippocampal_L",
    "ParaHippocampal_R",
    "Cingulum_Ant_L",
    "Cingulum_Ant_R",
    "Cingulum_Mid_L",
    "Cingulum_Mid_R",
    "Cingulum_Post_L",
    "Cingulum_Post_R",
    "Amygdala_L",
    "Amygdala_R"
]

In [5]:
controls_connectomes = [np.corrcoef(pd.read_csv(table)[alzheimer_features].T) for table in controls_path.iterdir() if plib.Path(table).stat().st_size > 15 * 1024]
patients_connectomes = [np.corrcoef(pd.read_csv(table)[alzheimer_features].T) for table in patients_path.iterdir() if plib.Path(table).stat().st_size > 15 * 1024]

**Смотрим на устойчивые гомологии фильтрации Вьеториса-Рипса**

Рассмотрим матрицу вида $(D)_{ij} = 1 - C_{ij},$ где $C$ - матрица коннектома, в качестве матрицы попарных расстояний на множестве точек. К каждой такой матрице применим фильтрацию Вьеториса-Рипса и построим для неё диаграмму устойчивости для гомологий размерности не более $2$.

In [ ]:
import ripser
import matplotlib.pyplot as plt

In [ ]:
rips = ripser.Rips(maxdim=2, verbose=False)
controls_vr_diagrams = []
patients_vr_diagrams = []

In [ ]:
%%time
for C in controls_connectomes:
    D = 1 - C
    controls_vr_diagrams.append(rips.fit_transform(D, distance_matrix=True))

for C in patients_connectomes:
    D = 1 - C
    patients_vr_diagrams.append(rips.fit_transform(D, distance_matrix=True))

In [ ]:
controls_vr_diagrams = np.array(controls_vr_diagrams)
patients_vr_diagrams = np.array(patients_vr_diagrams)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%%time

plt.figure(figsize=(6, 6))
plt.grid(True)
for diagram in controls_vr_diagrams:
    plt.scatter(
        [x[0] for x in diagram[1]],
        [x[1] for x in diagram[1]], # 1-dimensional homologies
        alpha=0.005, c='red')
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
plt.grid(True)
for diagram in patients_vr_diagrams:
    plt.scatter(
        [x[0] for x in diagram[1]],
        [x[1] for x in diagram[1]], # 1-dimensional homologies
        alpha=0.005, c='blue')

In [ ]:
plt.figure(figsize=(6, 6))
plt.grid(True)
for diagram in controls_vr_diagrams:
    plt.scatter(
        x=[x[0] for x in diagram[2]],
        y=[x[1] for x in diagram[2]], # 2-dimensional homologies
        alpha=0.005, c='orange')

In [ ]:
plt.figure(figsize=(6, 6))
plt.grid(True)
for diagram in patients_vr_diagrams:
    plt.scatter(
        [x[0] for x in diagram[2]],
        [x[1] for x in diagram[2]], # 2-dimensional homologies
        alpha=0.005, c=['purple'])

In [ ]:
d = 30
bins_left = np.arange(0, 2 * d)

In [ ]:
bins = []
for l in bins_left:
    for r in np.arange(l, 2 * d + 1, 1):
        bins.append((l / d, r / d))

In [ ]:
len(bins)

In [ ]:
bin_to_id = dict()
for i in range(len(bins)):
    bin_to_id[bins[i]] = i

In [ ]:
controls_vectors = []
for diagram in controls_vr_diagrams:
    vec = np.zeros((len(bins)))
    for point in diagram[1]:
        x, y = int(point[0] * d) / d, int(point[1] * d) / d
        vec[bin_to_id[(x, y)]] += 1
    for point in diagram[2]:
        x, y = int(point[0] * d) / d, int(point[1] * d) / d
        vec[bin_to_id[(x, y)]] += 1
    controls_vectors.append(vec)

In [ ]:
patients_vectors = []
for diagram in patients_vr_diagrams:
    vec = np.zeros((len(bins)))
    for point in diagram[1]:
        x, y = int(point[0] * d) / d, int(point[1] * d) / d
        vec[bin_to_id[(x, y)]] += 1
    for point in diagram[2]:
        x, y = int(point[0] * d) / d, int(point[1] * d) / d
        vec[bin_to_id[(x, y)]] += 1
    patients_vectors.append(vec)

In [ ]:
import sklearn.neighbors

In [ ]:
clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10)

In [ ]:
controls_vectors = np.array(controls_vectors)
patients_vectors = np.array(patients_vectors)

In [ ]:
X_train = np.concatenate((controls_vectors, patients_vectors), axis=0)
y_train = np.zeros(len(X_train))
y_train[len(controls_vectors):] = 1

In [ ]:
np.random.seed(57)
np.random.shuffle(X_train)
np.random.seed(57)
np.random.shuffle(y_train)

In [ ]:
clf = clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_train)

In [ ]:
np.sum(preds != y_train)

In [ ]:
thr = 0.5
def accuracy(estimator, X, y):
    return np.mean((estimator.predict(X) >= thr) == y)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(sklearn.neighbors.KNeighborsClassifier(n_neighbors=10), X_train, y_train, scoring=accuracy)

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression

In [ ]:
cross_val_score(sklearn.linear_model.LinearRegression(), X_train, y_train, scoring=accuracy)

In [ ]:
cross_val_score(sklearn.linear_model.Lasso(alpha=0.05), X_train, y_train, scoring=accuracy)

In [ ]:
cross_val_score(sklearn.linear_model.Ridge(alpha=0.1), X_train, y_train, scoring=accuracy)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
cross_val_score(sklearn.ensemble.RandomForestClassifier(n_estimators=100), X_train, y_train, scoring=accuracy)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
cross_val_score(sklearn.tree.DecisionTreeClassifier(), X_train, y_train, scoring=accuracy)

In [ ]:
X_train = np.concatenate((controls_vectors, patients_vectors), axis=0)
y_train = np.zeros(len(X_train))
y_train[len(controls_vectors):] = 1
np.random.seed(57)
np.random.shuffle(X_train)
np.random.seed(57)
np.random.shuffle(y_train)

In [ ]:
len(patients_vectors)

In [ ]:
from sklearn.model_selection import LeaveOneOut

In [ ]:
X = X_train
y = y_train

In [ ]:
%%time
for n_neighbors in range(50, 210, 10):
    clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=n_neighbors)
    score = 0
    for train_index, test_index in LeaveOneOut().split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        score += clf.predict(X_test) == y_test
    print(n_neighbors, score / len(X))

In [ ]:
cross_val_score(sklearn.linear_model.Lasso(alpha=0.1), X_train, y_train, scoring=accuracy)

In [ ]:
cross_val_score(sklearn.linear_model.Ridge(alpha=0.1), X_train, y_train, scoring=accuracy)

In [ ]:
cross_val_score(sklearn.tree.DecisionTreeClassifier(), X_train, y_train, scoring=accuracy)

## Строим базис пространства циклов и проецируем графы (векторы весов рёбер графов) объектов на это пространство, используем проекции как векторы признаков для обучения ##

### Описание идеи ###

Рассмотрим все рёбра, которые не попали в максимальное остовное дерево. Таких рёбер в полном графе ровно $$\frac{n(n - 1)}{2} - n + 1 = \frac{(n - 1)(n - 2)}{2}.$$

Каждое такое ребро при добавлении к максимальному остовному дереву порождает ровно один цикл, который является представителем своего класса $1$-мерных гомологий полного графа (поскольку мы не рассматриваем грани размерности выше $1$).

Для каждого ребра не из максимального остовного дерева возьмём вектор в пространстве линейных комбинаций рёбер графа, который соответствует циклу, порождённому этим ребром (этот вектор равен знакочередующейся сумме рёбер.  Всего такие векторы линейно независимы и составляют базис пространства циклов, которое является подпространством размерности $\frac{(n - 1)(n - 2)}{2}$ пространства линейных комбинаций рёбер (пространстве цепей размерности $1$), размерность которого составляет $\frac{n(n - 1)}{2}$.

Переберём все рёбра, не входящие в максимальный остов графа ```D_mean```, и для каждого ребра построим вектор, соответствующий циклу, который порождается добвлением этого ребра к остову. Таким образом, мы построим базис пространства циклов. Далее, будем представлять каждый коннектом из выборки как вектор весов рёбер (длины $n(n - 1) / 2$) и проецировать этот вектор на пространство циклов, а также находить разложение этой проекции по построенному нами выше базису пространства циклов.

У описанной процедуры есть простая геометрическая интерпретация: если для данного коннектома в разложении его проекции по базису коэффициент при каком-то цикле большой, то данный цикл хорошо выражен в данном коннектоме.

Предлагается использовать полученные разложения проекций коннектомов по построенному базису в виде векторов признаков для обучения моделей классификации.

### Алгоритм: ###

Находим поэлементное среднее матриц коннектомов всех объектов выборки:  

In [10]:
D_mean = 1 - np.mean(patients_connectomes, axis=0)

Напишем функцию, которая строит максимальное остовное дерево в графе алгоритмом Прима:

In [11]:
import typing as tp

In [12]:
def CalcMaxSpanningTreeEdgesSet(D: np.array) -> tp.List[tp.Tuple[int, int]]:
    """
    С помощью алгоритма Прима находим
    максимальное по весу остовное дерево в графе,
    который задан матрицей смежности D.
    
    Возвращаем список рёбер (пар вершин), 
    которые входят в максимальное остовное дерево.
    """
    n = len(D)
    assert D.shape == (n, n)
    
    dist = D[0].copy()
    prev = np.zeros(n)
        
    max_spanning_tree_edges = []
    
    for step in range(1, n):
        max_dist = None
        argmax = None
        for i in range(1, n):
            if dist[i] and (max_dist is None or max_dist < dist[i]):
                max_dist = dist[i]
                argmax = i
                
        p = int(prev[argmax])
                
        max_spanning_tree_edges.append((min(argmax, p), max(argmax, p)))
        dist[argmax] = 0
        
        for i in range(1, n):
            if dist[i] and dist[i] < D[argmax][i]:
                dist[i] = D[argmax][i]
                prev[i] = argmax
                
    max_spanning_tree_edges.sort()
    return max_spanning_tree_edges

Находим максимальное остовное дерево для среднего матриц коннектомов по контрольной группе:

In [13]:
%%time
D_mean_MaxST = CalcMaxSpanningTreeEdgesSet(D_mean)

CPU times: user 145 µs, sys: 0 ns, total: 145 µs
Wall time: 109 µs


По полученному набору рёбер остовного дерева построим матрицу смежности и списки смежности этого дерева (как невзвешенного графа).

In [14]:
D_mean_MaxST_adj_lists = [[] for i in range(len(D_mean))]
D_mean_MaxST_adj_matrix = np.zeros(D_mean.shape)

for edge in D_mean_MaxST:
    D_mean_MaxST_adj_lists[edge[0]].append(edge[1])
    D_mean_MaxST_adj_lists[edge[1]].append(edge[0])
    D_mean_MaxST_adj_matrix[edge[0]][edge[1]] = 1
    D_mean_MaxST_adj_matrix[edge[1]][edge[0]] = 1

Напишем несколько вспомогательных функций:

In [15]:
def Dfs(v: int, parent: tp.List[int], adj_lists: tp.List[tp.List[int]]) -> None:
    """
    Делаем обход дерева в глубину,
    при этом в массиве parent для каждой вершины
    сохраняем номер её предка. 
    """
    p = parent[v]
    for u in adj_lists[v]:
        if u != p:
            parent[u] = v
            Dfs(u, parent, adj_lists)
            

def GetEdgeNum(u: int, v: int, n: int) -> int:
    """
    Получаем по паре вершин u, v и общему числу вершин в графе n
    номер ребра (u, v) в графе
    (от 0 до n * (n - 1) // 2 - 1 включительно).
    """
    u, v = min(u, v), max(u, v)                 
    return u * (n * 2 - 1 - u) // 2 + v - u - 1

    
def GetPath(s: int, t: int, adj_lists: tp.List[tp.List[int]]) -> tp.List[int]:
    """
    Находим в дереве, которое задаётся списками смежности adj_lists,
    путь от вершины s до вершины t и возвращаем список вершин на этом пути.
    """
    parent = [-1] * len(adj_lists)
    Dfs(s, parent, adj_lists)
    path = []
    while t != s:
        path.append(t)
        t = parent[t]
    path.append(s)
    return path

Строим векторы, соответствующие циклам:

In [16]:
%%time

n = len(D_mean)
n_edges = n * (n - 1) // 2
cycle_eigenvectors = []

for i in range(n):
    for j in range(i + 1, n):
        if not D_mean_MaxST_adj_matrix[i, j]:
            cycle_eigenvector = np.zeros(n_edges)
            path = GetPath(i, j, D_mean_MaxST_adj_lists)
            for k in range(len(path)):
                cur, nxt = path[k], path[(k + 1) % len(path)]
                edge_index = GetEdgeNum(cur, nxt, n)
                cycle_eigenvector[edge_index] = 1 # if cur < nxt else -1
            cycle_eigenvector /= np.sqrt(len(path))
            cycle_eigenvectors.append(cycle_eigenvector)
            assert np.abs(cycle_eigenvector @ cycle_eigenvector - 1) < 0.0000001
            
cycle_eigenvectors = np.array(cycle_eigenvectors).T

CPU times: user 3.49 ms, sys: 594 µs, total: 4.08 ms
Wall time: 9.12 ms


Находим проекции $\pi_С(v_i),\ i = 1, \ldots, N$ (где $N$ - число объектов в выборке, $v_i$ - вектор весов рёбер для $i$-го объекта выборки) на пространство циклов $C$ по формуле
$$\pi_C(v) = C(C^TC)^{-1}C^Tv.$$

Нас интересуют именно коэффициенты в разложении $\pi_C(v)$ по базису из циклов, то есть, из столбцов матрицы $C$. Вектор этих коэфициентов, соответственно, находится по формуле $(C^TC)^{-1}C^Tv$.

Для начала строим матрицу проекции $\mathrm{Proj} = (C^TC)^{-1}C^T$:

In [17]:
import scipy.linalg

In [18]:
%%time
proj_matrix = scipy.linalg.inv(cycle_eigenvectors.T @ cycle_eigenvectors) @ cycle_eigenvectors.T

CPU times: user 2.87 ms, sys: 4.58 ms, total: 7.45 ms
Wall time: 10.3 ms


Напишем функцию для перевода объектов из представления в виде матрицы коннектома в представление в виде вектора весов рёбер (эквивалентно, делаем ```flatten()``` для части матрицы коннектома выше главной диагонали): 

In [19]:
def TransformConnectomeToEdgeVector(C: np.array) -> np.array:
    n = len(C)
    assert C.shape == (n, n)
    
    n_edges = n * (n - 1) // 2
    edge_vector = np.ones(n_edges)
    
    l = 0
    for i in range(n):
        edge_vector[l:l + n - i - 1] -= C[i, i + 1:]
        l += n - i - 1
    return edge_vector

Находим разложения по базису циклов для проекций коннектомов из контрольной группы:

In [20]:
controls_edge_vectors = []
for C in controls_connectomes:
    controls_edge_vectors.append(TransformConnectomeToEdgeVector(C))
    
controls_edge_vectors = np.array(controls_edge_vectors).T

In [21]:
controls_cycle_projections = proj_matrix @ controls_edge_vectors
controls_cycle_projections = controls_cycle_projections.T # чтобы получить матрицу размера N_controls x ((n - 1)(n - 2)/2)

In [22]:
controls_cycle_projections.shape

(1295, 55)

Проделаем то же самое для коннектомов из группы пациентов:

In [23]:
patients_edge_vectors = []
for C in patients_connectomes:
    patients_edge_vectors.append(TransformConnectomeToEdgeVector(C))
    
patients_edge_vectors = np.array(patients_edge_vectors).T

In [24]:
patients_cycle_projections = proj_matrix @ patients_edge_vectors
patients_cycle_projections = patients_cycle_projections.T # чтобы получить матрицу размера N_patients x ((n - 1)(n - 2)/2)

Строим датасет на полученных признаках для обучения (```X```, ```y```):

In [25]:
X = np.concatenate((controls_cycle_projections, patients_cycle_projections), axis=0)
y = np.zeros(len(X))
y[len(controls_cycle_projections):] = 1
np.random.seed(57)
np.random.shuffle(X)
np.random.seed(57)
np.random.shuffle(y)

Также построим датасет на векторизованных исходных матрицах коннектомов (```X_raw```, ```y_raw```):

In [ ]:
patients_edge_vectorsT = patients_edge_vectors.T
controls_edge_vectorsT = controls_edge_vectors.T

X_raw = np.concatenate((controls_edge_vectorsT, patients_edge_vectorsT), axis=0)
y_raw = np.zeros(len(X_raw))
y_raw[len(controls_edge_vectorsT):] = 1
np.random.seed(57)
np.random.shuffle(X)
np.random.seed(57)
np.random.shuffle(y)

#### Пытаемся обучить KNN и сравнить качество на проекциях на циклы vs векторизованных исходных матрицах коннектомов:

In [44]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, LeaveOneOut

In [46]:
n_neighbors_variants = [1, 3, 5, 11, 21, 31, 51, 71, 101, 151, 201]

In [47]:
%%time
# проекции на циклы
for n_neighbors in n_neighbors_variants:
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    score = 0
    for train_index, test_index in LeaveOneOut().split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        score += clf.predict(X_test) == y_test
    print("KNN leave-one-out accuracy for {} neighbors on cycle projections:".format(n_neighbors), score / len(X))

KNN leave-one-out accuracy for 1 neighbors on cycle projections: [0.55734597]
KNN leave-one-out accuracy for 3 neighbors on cycle projections: [0.55876777]
KNN leave-one-out accuracy for 5 neighbors on cycle projections: [0.56919431]
KNN leave-one-out accuracy for 11 neighbors on cycle projections: [0.59194313]
KNN leave-one-out accuracy for 21 neighbors on cycle projections: [0.60758294]
KNN leave-one-out accuracy for 31 neighbors on cycle projections: [0.61232227]
KNN leave-one-out accuracy for 51 neighbors on cycle projections: [0.62085308]
KNN leave-one-out accuracy for 71 neighbors on cycle projections: [0.61800948]
KNN leave-one-out accuracy for 101 neighbors on cycle projections: [0.62843602]
KNN leave-one-out accuracy for 151 neighbors on cycle projections: [0.62606635]
KNN leave-one-out accuracy for 201 neighbors on cycle projections: [0.62322275]
CPU times: user 2min 32s, sys: 7min 2s, total: 9min 35s
Wall time: 52.2 s


Сравним с обучением на непосредственно векторизованных матрицах коннектомов:

In [48]:
%%time
# векторизованные матрицы коннектомов
for n_neighbors in n_neighbors_variants:
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    score = 0
    for train_index, test_index in LeaveOneOut().split(X_raw):
        X_train, X_test = X_raw[train_index], X_raw[test_index]
        y_train, y_test = y_raw[train_index], y_raw[test_index]
        clf.fit(X_train, y_train)
        score += clf.predict(X_test) == y_test
    print("KNN leave-one-out accuracy for {} neighbors on vectorized connectomes:".format(n_neighbors), score / len(X))

KNN leave-one-out accuracy for 1 neighbors on vectorized connectomes: [0.57393365]
KNN leave-one-out accuracy for 3 neighbors on vectorized connectomes: [0.57251185]
KNN leave-one-out accuracy for 5 neighbors on vectorized connectomes: [0.59905213]
KNN leave-one-out accuracy for 11 neighbors on vectorized connectomes: [0.61137441]
KNN leave-one-out accuracy for 21 neighbors on vectorized connectomes: [0.63033175]
KNN leave-one-out accuracy for 31 neighbors on vectorized connectomes: [0.62132701]
KNN leave-one-out accuracy for 51 neighbors on vectorized connectomes: [0.63175355]
KNN leave-one-out accuracy for 71 neighbors on vectorized connectomes: [0.63033175]
KNN leave-one-out accuracy for 101 neighbors on vectorized connectomes: [0.62748815]
KNN leave-one-out accuracy for 151 neighbors on vectorized connectomes: [0.63459716]
KNN leave-one-out accuracy for 201 neighbors on vectorized connectomes: [0.63033175]
CPU times: user 2min 40s, sys: 7min 30s, total: 10min 11s
Wall time: 55.9 s


#### Сравним на линейных моделях:

In [52]:
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.svm import SVC

In [64]:
%%time
# проекции на циклы
clf = LogisticRegression(max_iter=500)
score = 0
for train_index, test_index in LeaveOneOut().split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    score += clf.predict(X_test) == y_test
print(score / len(X))

[0.61421801]
CPU times: user 8min 59s, sys: 22min 14s, total: 31min 13s
Wall time: 2min 52s


In [65]:
%%time
# векторизованные исходные матрицы
clf = LogisticRegression(max_iter=500)
score = 0
for train_index, test_index in LeaveOneOut().split(X_raw):
    X_train, X_test = X_raw[train_index], X_raw[test_index]
    y_train, y_test = y_raw[train_index], y_raw[test_index]
    clf.fit(X_train, y_train)
    score += clf.predict(X_test) == y_test
print(score / len(X))

[0.61800948]
CPU times: user 12min 9s, sys: 29min 56s, total: 42min 6s
Wall time: 3min 54s


In [58]:
%%time

clf = SVC()

score = 0
for train_index, test_index in LeaveOneOut().split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    score += clf.predict(X_test) == y_test
print(score / len(X))

[0.62037915]
CPU times: user 6min 52s, sys: 154 ms, total: 6min 52s
Wall time: 6min 52s


In [ ]:
%%time

clf = SVC()

score = 0
for train_index, test_index in LeaveOneOut().split(X_raw):
    X_train, X_test = X_raw[train_index], X_raw[test_index]
    y_train, y_test = y_raw[train_index], y_raw[test_index]
    clf.fit(X_train, y_train)
    score += clf.predict(X_test) == y_test
print(score / len(X))